In [1]:
import numpy as np
from scipy import optimize
from scipy import interpolate
import matplotlib.pyplot as plt
import time

In [3]:
def prob0(alpha, beta):
    """
    Input: value of alpha (positive or negative) in which we are measuring and value of the beta we are applying
    Output: the probability of findind 0 photons
    """
    return np.exp(-(alpha+beta)**2)

def prob0_prior(absalpha, beta, prior):
    """
    Input: absolute value of the alpha we are measuring, beta we are applying and the probability (prior) that alpha is positive
    Output: the probability of finding 0 photons with the given prior
    """
    return prob0(absalpha, beta)*prior+prob0(-absalpha, beta)*(1-prior)

def update_prior(alpha, beta, prior, c):
    """
    Input: alpha in which we measure (abs), beta applying, the prior of positive alpha, and the result of measurement (0 or 1)
    Output: the new prior of having positive alpha given the result of the measurement
    """
    if c==0:
        return prob0(alpha, beta)*prior/prob0_prior(alpha, beta, prior)
    elif c==1:
        return (1-prob0(alpha, beta))*prior/(1-prob0_prior(alpha, beta, prior))
    else:
        raise ValueError

In [4]:
def interpol_from0to1(alpha, prior,  interpolation):
    """
    Input: alpha is the amplitude, prior is the probability of being +alpha and interpolation is function of two variables
    The interpolation of the funcion S is done in the interval of probabilities [0.5, 1)
    This function allows to use the interpolation function in the interval (0, 1)
    """
    if prior > 1:
        print('Error')
        prior = 1       
    if prior >= 0.5:
        return interpolation(prior, alpha)[0]
    else:
        prior = 1 - prior
        return interpolation(prior, alpha)[0]


def auxiliar_S_func(vector, prior, alpha, S_interpolation):
    """
    Input: vector = [beta, theta] ----> what will be optimized, prior of +alpha, value of the amplitude alpha, and the
    interpolation of alpha
    Output: The value of the function of the dynamic programming recurrence relation
    """
    
    eta0 = update_prior(alpha*np.sin(vector[1]), vector[0], prior, 0)
    eta1 = update_prior(alpha*np.sin(vector[1]), vector[0], prior, 1)
    
    x = prob0_prior(alpha*np.sin(vector[1]), vector[0], prior)*interpol_from0to1(alpha*np.cos(vector[1]), eta0, S_interpolation)+(1-prob0_prior(alpha*np.sin(vector[1]), vector[0], prior))*interpol_from0to1(alpha*np.cos(vector[1]), eta1, S_interpolation)
    return -x # since we only know how to minimize

def auxiliar_S_func2(beta, prior, alpha, S_interpolation):
    """
    Input: beta, what will be optimized. prior of +alpha, alpha and the interpolation of S
    Output: The value of the function of the dynamic programming recurrence relation for theta already optimized
            and fixed to np.pi/2
    """
    eta0 = update_prior(alpha, beta, prior, 0)
    eta1 = update_prior(alpha, beta, prior, 1)
    
    x = prob0_prior(alpha, beta, prior)*interpol_from0to1(0, eta0, S_interpolation)+(1-prob0_prior(alpha, beta, prior))*interpol_from0to1(0, eta1, S_interpolation)
    return -x # since we only know how to minimize


def optimization(N, Nprob, Nalpha, alpha_max):
    """
    Input: the number of stages, N. The number of discretization in eta (probabilities) and alphas.
    Output: Saves .npy documents with the optimal betas, thetas and success probabilities.
    """
    
    start = time.time()
    
    etas = np.linspace(0.5, 1, Nprob) # etas from 0.5 to 1
    alphas = np.linspace(0, alpha_max, Nalpha) # alphas from 0 to alpha_max

    np.save('alphas.npy', alphas)
    np.save('etas.npy', etas)
    
    S = np.zeros((N+1, Nalpha, Nprob)) # matrix that saves the values of the objective function
    
    thetas_res = np.zeros((N, Nalpha, Nprob)) # matrix that saves the optimal attenuation for each value of alpha and beta at
                                              # each step
    betas_res = np.zeros((N, Nalpha, Nprob))  # matrix that saves the optimal displacement for each value of alpha and beta at
                                              # each step
    
    # We know how S is for the last step:
    for n_alpha in range(Nalpha):
        for n_eta in range(Nprob):
            S[N, n_alpha, n_eta] = etas[n_eta]
    

    i=-1
    # loop for all the stages
    for n in range(N-1, -1, -1):
        i+=1
        S_interp = interpolate.interp2d(etas, alphas, S[n+1], kind='cubic') # 2d interpolation
        bnds = [(None, None), (0, np.pi/2-0.001)]
        
        # since for the last step it is known that theta = np.pi/2 is optimum (just to avoid some calculations).
        if i==0:
            for a in range(Nalpha):
                for b in range(Nprob):
                    if a==0:
                        S[n,a,b] = S[n+1,a,b]
                        betas_res[n, a, b] = None
                        thetas_res[n, a, b] = None                    
                    elif b==0 and a==1:
                        optimization = optimize.minimize(auxiliar_S_func2, -0.7, (etas[b], alphas[a], S_interp))
                        S[n, a, b] = -optimization.fun
                        betas_res[n, a, b] = optimization.x[0]
                        thetas_res[n, a, b] = np.pi/2
                    elif b!=0:
                        optimization = optimize.minimize(auxiliar_S_func2, betas_res[n, a, b-1], (etas[b], alphas[a], S_interp))
                        S[n, a, b] = -optimization.fun
                        betas_res[n, a, b] = optimization.x[0]
                        thetas_res[n, a, b] = np.pi/2
                    elif a!=1:
                        optimization = optimize.minimize(auxiliar_S_func2, betas_res[n, a-1, b], (etas[b], alphas[a], S_interp))
                        S[n, a, b] = -optimization.fun
                        betas_res[n, a, b] = optimization.x[0]
                        thetas_res[n, a, b] = np.pi/2
        
        # the other steps, optimizing over theta and beta
        else:
            for a in range(Nalpha):
                for b in range(Nprob):
                    if b == Nprob - 1: #prior = 1
                        S[n,a,b] = 1
                        betas_res[n, a, b] = None
                        thetas_res[n, a, b] = None                         
                    elif a==0:
                        S[n,a,b] = S[n+1,a,b]
                        betas_res[n, a, b] = None
                        thetas_res[n, a, b] = None                      
                    elif b==0 and a==1:
                        optimization = optimize.minimize(auxiliar_S_func, [-0.7, np.pi/4], (etas[b], alphas[a], S_interp), bounds=bnds)
                        S[n, a, b] = -optimization.fun
                        betas_res[n, a, b] = optimization.x[0]
                        thetas_res[n, a, b] = optimization.x[1]
                    elif b!=0:
                        optimization = optimize.minimize(auxiliar_S_func, [betas_res[n, a, b-1], thetas_res[n, a, b-1]], (etas[b], alphas[a], S_interp), bounds=bnds)
                        S[n, a, b] = -optimization.fun
                        betas_res[n, a, b] = optimization.x[0]
                        thetas_res[n, a, b] = optimization.x[1]
                    elif a!=1:
                        optimization = optimize.minimize(auxiliar_S_func, [betas_res[n, a-1, b], thetas_res[n, a-1, b]], (etas[b], alphas[a], S_interp), bounds=bnds)
                        S[n, a, b] = -optimization.fun
                        betas_res[n, a, b] = optimization.x[0]
                        thetas_res[n, a, b] = optimization.x[1]

    
    np.save('results_thetas.npy', thetas_res)
    np.save('results_betas.npy', betas_res)
    np.save('S_matrix.npy', S)

    
    end = time.time()
    print(end - start)

N = 8   # number of layers


Nprob = 2000 # discretization of probabilities
Nalpha = 2000 # dicretization of alphas

    
    
optimization(N, Nprob, Nalpha, 1.5)

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 178
0 179
0 180
0 181
0 182
0 183
0 184


0 1329
0 1330
0 1331
0 1332
0 1333
0 1334
0 1335
0 1336
0 1337
0 1338
0 1339
0 1340
0 1341
0 1342
0 1343
0 1344
0 1345
0 1346
0 1347
0 1348
0 1349
0 1350
0 1351
0 1352
0 1353
0 1354
0 1355
0 1356
0 1357
0 1358
0 1359
0 1360
0 1361
0 1362
0 1363
0 1364
0 1365
0 1366
0 1367
0 1368
0 1369
0 1370
0 1371
0 1372
0 1373
0 1374
0 1375
0 1376
0 1377
0 1378
0 1379
0 1380
0 1381
0 1382
0 1383
0 1384
0 1385
0 1386
0 1387
0 1388
0 1389
0 1390
0 1391
0 1392
0 1393
0 1394
0 1395
0 1396
0 1397
0 1398
0 1399
0 1400
0 1401
0 1402
0 1403
0 1404
0 1405
0 1406
0 1407
0 1408
0 1409
0 1410
0 1411
0 1412
0 1413
0 1414
0 1415
0 1416
0 1417
0 1418
0 1419
0 1420
0 1421
0 1422
0 1423
0 1424
0 1425
0 1426
0 1427
0 1428
0 1429
0 1430
0 1431
0 1432
0 1433
0 1434
0 1435
0 1436
0 1437
0 1438
0 1439
0 1440
0 1441
0 1442
0 1443
0 1444
0 1445
0 1446
0 1447
0 1448
0 1449
0 1450
0 1451
0 1452
0 1453
0 1454
0 1455
0 1456
0 1457
0 1458
0 1459
0 1460
0 1461
0 1462
0 1463
0 1464
0 1465
0 1466
0 1467
0 1468
0 1469
0 1470
0 1471

1 601
1 602
1 603
1 604
1 605
1 606
1 607
1 608
1 609
1 610
1 611
1 612
1 613
1 614
1 615
1 616
1 617
1 618
1 619
1 620
1 621
1 622
1 623
1 624
1 625
1 626
1 627
1 628
1 629
1 630
1 631
1 632
1 633
1 634
1 635
1 636
1 637
1 638
1 639
1 640
1 641
1 642
1 643
1 644
1 645
1 646
1 647
1 648
1 649
1 650
1 651
1 652
1 653
1 654
1 655
1 656
1 657
1 658
1 659
1 660
1 661
1 662
1 663
1 664
1 665
1 666
1 667
1 668
1 669
1 670
1 671
1 672
1 673
1 674
1 675
1 676
1 677
1 678
1 679
1 680
1 681
1 682
1 683
1 684
1 685
1 686
1 687
1 688
1 689
1 690
1 691
1 692
1 693
1 694
1 695
1 696
1 697
1 698
1 699
1 700
1 701
1 702
1 703
1 704
1 705
1 706
1 707
1 708
1 709
1 710
1 711
1 712
1 713
1 714
1 715
1 716
1 717
1 718
1 719
1 720
1 721
1 722
1 723
1 724
1 725
1 726
1 727
1 728
1 729
1 730
1 731
1 732
1 733
1 734
1 735
1 736
1 737
1 738
1 739
1 740
1 741
1 742
1 743
1 744
1 745
1 746
1 747
1 748
1 749
1 750
1 751
1 752
1 753
1 754
1 755
1 756
1 757
1 758
1 759
1 760
1 761
1 762
1 763
1 764
1 765
1 766
1 76

1 1829
1 1830
1 1831
1 1832
1 1833
1 1834
1 1835
1 1836
1 1837
1 1838
1 1839
1 1840
1 1841
1 1842
1 1843
1 1844
1 1845
1 1846
1 1847
1 1848
1 1849
1 1850
1 1851
1 1852
1 1853
1 1854
1 1855
1 1856
1 1857
1 1858
1 1859
1 1860
1 1861
1 1862
1 1863
1 1864
1 1865
1 1866
1 1867
1 1868
1 1869
1 1870
1 1871
1 1872
1 1873
1 1874
1 1875
1 1876
1 1877
1 1878
1 1879
1 1880
1 1881
1 1882
1 1883
1 1884
1 1885
1 1886
1 1887
1 1888
1 1889
1 1890
1 1891
1 1892
1 1893
1 1894
1 1895
1 1896
1 1897
1 1898
1 1899
1 1900
1 1901
1 1902
1 1903
1 1904
1 1905
1 1906
1 1907
1 1908
1 1909
1 1910
1 1911
1 1912
1 1913
1 1914
1 1915
1 1916
1 1917
1 1918
1 1919
1 1920
1 1921
1 1922
1 1923
1 1924
1 1925
1 1926
1 1927
1 1928
1 1929
1 1930
1 1931
1 1932
1 1933
1 1934
1 1935
1 1936
1 1937
1 1938
1 1939
1 1940
1 1941
1 1942
1 1943
1 1944
1 1945
1 1946
1 1947
1 1948
1 1949
1 1950
1 1951
1 1952
1 1953
1 1954
1 1955
1 1956
1 1957
1 1958
1 1959
1 1960
1 1961
1 1962
1 1963
1 1964
1 1965
1 1966
1 1967
1 1968
1 1969
1 1970
1 1971

2 1158
2 1159
2 1160
2 1161
2 1162
2 1163
2 1164
2 1165
2 1166
2 1167
2 1168
2 1169
2 1170
2 1171
2 1172
2 1173
2 1174
2 1175
2 1176
2 1177
2 1178
2 1179
2 1180
2 1181
2 1182
2 1183
2 1184
2 1185
2 1186
2 1187
2 1188
2 1189
2 1190
2 1191
2 1192
2 1193
2 1194
2 1195
2 1196
2 1197
2 1198
2 1199
2 1200
2 1201
2 1202
2 1203
2 1204
2 1205
2 1206
2 1207
2 1208
2 1209
2 1210
2 1211
2 1212
2 1213
2 1214
2 1215
2 1216
2 1217
2 1218
2 1219
2 1220
2 1221
2 1222
2 1223
2 1224
2 1225
2 1226
2 1227
2 1228
2 1229
2 1230
2 1231
2 1232
2 1233
2 1234
2 1235
2 1236
2 1237
2 1238
2 1239
2 1240
2 1241
2 1242
2 1243
2 1244
2 1245
2 1246
2 1247
2 1248
2 1249
2 1250
2 1251
2 1252
2 1253
2 1254
2 1255
2 1256
2 1257
2 1258
2 1259
2 1260
2 1261
2 1262
2 1263
2 1264
2 1265
2 1266
2 1267
2 1268
2 1269
2 1270
2 1271
2 1272
2 1273
2 1274
2 1275
2 1276
2 1277
2 1278
2 1279
2 1280
2 1281
2 1282
2 1283
2 1284
2 1285
2 1286
2 1287
2 1288
2 1289
2 1290
2 1291
2 1292
2 1293
2 1294
2 1295
2 1296
2 1297
2 1298
2 1299
2 1300

...

5 523
5 524
5 525
5 526
5 527
5 528
5 529
5 530
5 531
5 532
5 533
5 534
5 535
5 536
5 537
5 538
5 539
5 540
5 541
5 542
5 543
5 544
5 545
5 546
5 547
5 548
5 549
5 550
5 551
5 552
5 553
5 554
5 555
5 556
5 557
5 558
5 559
5 560
5 561
5 562
5 563
5 564
5 565
5 566
5 567
5 568
5 569
5 570
5 571
5 572
5 573
5 574
5 575
5 576
5 577
5 578
5 579
5 580
5 581
5 582
5 583
5 584
5 585
5 586
5 587
5 588
5 589
5 590
5 591
5 592
5 593
5 594
5 595
5 596
5 597
5 598
5 599
5 600
5 601
5 602
5 603
5 604
5 605
5 606
5 607
5 608
5 609
5 610
5 611
5 612
5 613
5 614
5 615
5 616
5 617
5 618
5 619
5 620
5 621
5 622
5 623
5 624
5 625
5 626
5 627
5 628
5 629
5 630
5 631
5 632
5 633
5 634
5 635
5 636
5 637
5 638
5 639
5 640
5 641
5 642
5 643
5 644
5 645
5 646
5 647
5 648
5 649
5 650
5 651
5 652
5 653
5 654
5 655
5 656
5 657
5 658
5 659
5 660
5 661
5 662
5 663
5 664
5 665
5 666
5 667
5 668
5 669
5 670
5 671
5 672
5 673
5 674
5 675
5 676
5 677
5 678
5 679
5 680
5 681
5 682
5 683
5 684
5 685
5 686
5 687
5 688
5 68

5 1762
5 1763
5 1764
5 1765
5 1766
5 1767
5 1768
5 1769
5 1770
5 1771
5 1772
5 1773
5 1774
5 1775
5 1776
5 1777
5 1778
5 1779
5 1780
5 1781
5 1782
5 1783
5 1784
5 1785
5 1786
5 1787
5 1788
5 1789
5 1790
5 1791
5 1792
5 1793
5 1794
5 1795
5 1796
5 1797
5 1798
5 1799
5 1800
5 1801
5 1802
5 1803
5 1804
5 1805
5 1806
5 1807
5 1808
5 1809
5 1810
5 1811
5 1812
5 1813
5 1814
5 1815
5 1816
5 1817
5 1818
5 1819
5 1820
5 1821
5 1822
5 1823
5 1824
5 1825
5 1826
5 1827
5 1828
5 1829
5 1830
5 1831
5 1832
5 1833
5 1834
5 1835
5 1836
5 1837
5 1838
5 1839
5 1840
5 1841
5 1842
5 1843
5 1844
5 1845
5 1846
5 1847
5 1848
5 1849
5 1850
5 1851
5 1852
5 1853
5 1854
5 1855
5 1856
5 1857
5 1858
5 1859
5 1860
5 1861
5 1862
5 1863
5 1864
5 1865
5 1866
5 1867
5 1868
5 1869
5 1870
5 1871
5 1872
5 1873
5 1874
5 1875
5 1876
5 1877
5 1878
5 1879
5 1880
5 1881
5 1882
5 1883
5 1884
5 1885
5 1886
5 1887
5 1888
5 1889
5 1890
5 1891
5 1892
5 1893
5 1894
5 1895
5 1896
5 1897
5 1898
5 1899
5 1900
5 1901
5 1902
5 1903
5 1904

6 1091
6 1092
6 1093
6 1094
6 1095
6 1096
6 1097
6 1098
6 1099
6 1100
6 1101
6 1102
6 1103
6 1104
6 1105
6 1106
6 1107
6 1108
6 1109
6 1110
6 1111
6 1112
6 1113
6 1114
6 1115
6 1116
6 1117
6 1118
6 1119
6 1120
6 1121
6 1122
6 1123
6 1124
6 1125
6 1126
6 1127
6 1128
6 1129
6 1130
6 1131
6 1132
6 1133
6 1134
6 1135
6 1136
6 1137
6 1138
6 1139
6 1140
6 1141
6 1142
6 1143
6 1144
6 1145
6 1146
6 1147
6 1148
6 1149
6 1150
6 1151
6 1152
6 1153
6 1154
6 1155
6 1156
6 1157
6 1158
6 1159
6 1160
6 1161
6 1162
6 1163
6 1164
6 1165
6 1166
6 1167
6 1168
6 1169
6 1170
6 1171
6 1172
6 1173
6 1174
6 1175
6 1176
6 1177
6 1178
6 1179
6 1180
6 1181
6 1182
6 1183
6 1184
6 1185
6 1186
6 1187
6 1188
6 1189
6 1190
6 1191
6 1192
6 1193
6 1194
6 1195
6 1196
6 1197
6 1198
6 1199
6 1200
6 1201
6 1202
6 1203
6 1204
6 1205
6 1206
6 1207
6 1208
6 1209
6 1210
6 1211
6 1212
6 1213
6 1214
6 1215
6 1216
6 1217
6 1218
6 1219
6 1220
6 1221
6 1222
6 1223
6 1224
6 1225
6 1226
6 1227
6 1228
6 1229
6 1230
6 1231
6 1232
6 1233

7 324
7 325
7 326
7 327
7 328
7 329
7 330
7 331
7 332
7 333
7 334
7 335
7 336
7 337
7 338
7 339
7 340
7 341
7 342
7 343
7 344
7 345
7 346
7 347
7 348
7 349
7 350
7 351
7 352
7 353
7 354
7 355
7 356
7 357
7 358
7 359
7 360
7 361
7 362
7 363
7 364
7 365
7 366
7 367
7 368
7 369
7 370
7 371
7 372
7 373
7 374
7 375
7 376
7 377
7 378
7 379
7 380
7 381
7 382
7 383
7 384
7 385
7 386
7 387
7 388
7 389
7 390
7 391
7 392
7 393
7 394
7 395
7 396
7 397
7 398
7 399
7 400
7 401
7 402
7 403
7 404
7 405
7 406
7 407
7 408
7 409
7 410
7 411
7 412
7 413
7 414
7 415
7 416
7 417
7 418
7 419
7 420
7 421
7 422
7 423
7 424
7 425
7 426
7 427
7 428
7 429
7 430
7 431
7 432
7 433
7 434
7 435
7 436
7 437
7 438
7 439
7 440
7 441
7 442
7 443
7 444
7 445
7 446
7 447
7 448
7 449
7 450
7 451
7 452
7 453
7 454
7 455
7 456
7 457
7 458
7 459
7 460
7 461
7 462
7 463
7 464
7 465
7 466
7 467
7 468
7 469
7 470
7 471
7 472
7 473
7 474
7 475
7 476
7 477
7 478
7 479
7 480
7 481
7 482
7 483
7 484
7 485
7 486
7 487
7 488
7 489
7 49

7 1591
7 1592
7 1593
7 1594
7 1595
7 1596
7 1597
7 1598
7 1599
7 1600
7 1601
7 1602
7 1603
7 1604
7 1605
7 1606
7 1607
7 1608
7 1609
7 1610
7 1611
7 1612
7 1613
7 1614
7 1615
7 1616
7 1617
7 1618
7 1619
7 1620
7 1621
7 1622
7 1623
7 1624
7 1625
7 1626
7 1627
7 1628
7 1629
7 1630
7 1631
7 1632
7 1633
7 1634
7 1635
7 1636
7 1637
7 1638
7 1639
7 1640
7 1641
7 1642
7 1643
7 1644
7 1645
7 1646
7 1647
7 1648
7 1649
7 1650
7 1651
7 1652
7 1653
7 1654
7 1655
7 1656
7 1657
7 1658
7 1659
7 1660
7 1661
7 1662
7 1663
7 1664
7 1665
7 1666
7 1667
7 1668
7 1669
7 1670
7 1671
7 1672
7 1673
7 1674
7 1675
7 1676
7 1677
7 1678
7 1679
7 1680
7 1681
7 1682
7 1683
7 1684
7 1685
7 1686
7 1687
7 1688
7 1689
7 1690
7 1691
7 1692
7 1693
7 1694
7 1695
7 1696
7 1697
7 1698
7 1699
7 1700
7 1701
7 1702
7 1703
7 1704
7 1705
7 1706
7 1707
7 1708
7 1709
7 1710
7 1711
7 1712
7 1713
7 1714
7 1715
7 1716
7 1717
7 1718
7 1719
7 1720
7 1721
7 1722
7 1723
7 1724
7 1725
7 1726
7 1727
7 1728
7 1729
7 1730
7 1731
7 1732
7 1733